# Deployment

## API

In [12]:
%%writefile predict.py

# Import necessary modules
from flask import Flask, request, jsonify
import pickle
import pandas as pd

import traceback

# Create an instance of Flask class
app = Flask(__name__)

# Use pickle to load in the pre-trained model.
with open('model_rf.pkl', 'rb') as m:
    model_rf = pickle.load(m)

# Use pickle to load in the feature names.
with open('model_features.pkl', 'rb') as f:
    features = pickle.load(f)

@app.route('/')
def index():
    return 'Server is up and running!'

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        try:
            # Get data for prediction (Parse data as JSON)
            json_data = request.get_json()
            
            # Raise an Error if there are values not entered
            if not all(k in json_data for k in ["hp", "age", "km", "model"]):
                raise ValueError("Not enough data to make a prediction!")
            
            # Create a DataFrame from JSON
            df = pd.DataFrame.from_dict([json_data], orient='columns')
            
            # Create dummy variables and persist features
            df = pd.get_dummies(df).reindex(columns=features, fill_value=0)
            # Note that if a column name (value) in JSON is wrong, 
            # The relevant column is filled with '0'
            
            # Predict the target
            prediction = list(model_rf.predict(df))
            
            # Return the result in JSON format
            return jsonify({"prediction": prediction})
        
        except:
            return jsonify({'trace': traceback.format_exc()})
    else:
        print("Nothing posted!")
        return "Nothing posted!"

if __name__ == '__main__':
    app.run(debug=True)
# Debugging mode is enabled to provide code reloading and better error messages.

Overwriting predict.py


## How to Request/POST

In [2]:
#%%writefile request.py
import requests
url = 'http://localhost:5000/predict'
r = requests.post(url, json={"hp":"100", "age":"2", "km":"10000", "model":"Corsa"})
r.json()

{'prediction': [10599.177919347703]}

In [3]:
!curl localhost:5000/predict -d '{"hp":"100", "age":"2", "km":"10000", "model":"Corsa"}' -H 'Content-Type: application/json'
#-d --data: Send specified data in POST request
#-H, --header: Headers to supply with request

{
  "prediction": [
    10599.177919347703
  ]
}


In [1]:
!curl https://gutkyle.pythonanywhere.com/predict -d '{"hp":"100", "age":"2", "km":"10000", "model":"Corsa"}' -H 'Content-Type: application/json'


{"prediction":[10599.177919347703]}


In [4]:
%%writefile data.json
{
    "hp":"100",
    "age":"2",
    "km":"10000",
    "model":"Corsa"
}

Overwriting data.json


In [5]:
!curl -X POST localhost:5000/predict -d '@data.json' -H 'Content-Type: application/json'

{
  "prediction": [
    10599.177919347703
  ]
}


In [6]:
# !http http://localhost:5000/predict json={'hp':'100'}
# Requires HTTPie

## Test Results

In [7]:
import pickle
import pandas as pd

with open('model_rf.pkl', 'rb') as m:
    model_rf = pickle.load(m)
    
# Import imputed data
df_no_dummy = pd.read_pickle("autos_clean_no_dummy_20190805.pkl")

# Create features ann target
X = pd.get_dummies(df_no_dummy[['hp', 'age', 'km', 'model']]) # features
y = df_no_dummy['price'] # target

df = df_no_dummy[['hp', 'age', 'km', 'model', 'price']].copy()
df['prediction'] = model_rf.predict(X)

In [8]:
df.sample(5)

,hp,age,km,model,price,prediction
13129,87.0,1.0,20200.0,Clio,15650,15765.558460
7594,110.0,0.0,10.0,Astra,25365,21553.017866
5840,100.0,2.0,30600.0,Astra,15492,15391.644049
4647,81.0,1.0,13500.0,A3,22900,22433.526813
10698,100.0,3.0,55775.0,Insignia,14950,15007.863167


In [9]:
import requests

json_d = {
    "hp": 66,
    "age": 1,
    "km": 17000,
    "model": "Clio"
}

url = 'http://localhost:5000/predict'
r = requests.post(url, json = json_d)
r.json()

{'prediction': [11011.501326226444]}

In [3]:
import requests

json_d = {
    "hp": 66,
    "age": 1,
    "km": 17000,
    "model": "Clio"
}

url = 'https://gutkyle.pythonanywhere.com/predict'
r = requests.post(url, json = json_d)
r.json()

{'trace': 'Traceback (most recent call last):\n  File "/home/gutkyle/mysite/flask_app.py", line 32, in predict\n    raise ValueError("Not enough data to make a prediction!")\nValueError: Not enough data to make a prediction!\n'}

## Test Code

In [10]:
import pickle
import pandas as pd

with open('model_rf.pkl', 'rb') as m:
    model_rf = pickle.load(m)

with open('model_features.pkl', 'rb') as f:
    features = pickle.load(f)

json_d = {
    "hp": 100,
    "age": 3,
    "km": 10000,
    "model": "Corsa"
}


# Create a DataFrame from JSON
df_dene = pd.DataFrame.from_dict([json_d], orient='columns')

df_dene

,hp,age,km,model
0,100,3,10000,Corsa


In [11]:
# Create dummy variables and persist features
df_dene = pd.get_dummies(df_dene).reindex(columns=features, fill_value=0)
df_dene

,hp,age,km,model_A1,model_A3,model_Astra,model_Clio,model_Corsa,model_Espace,model_Insignia
0,100,3,10000,0,0,0,0,1,0,0
